In [6]:
%store -r test_set

In [7]:
test_set

,Argument ID,Conclusion,Stance,Premise,Argument ID,premise_tokenized,Openness to change,Self-Enhancement,Conservation,Self-Transcendence,general_label
0,A26004,We should end affirmative action,against,affirmative action helps with employment equity.,A26004,"[affirmative, action, helps, with, employment,...",0,1,1,2,"[0.0, 0.25, 0.25, 0.5]"
1,A26010,We should end affirmative action,in favor of,affirmative action can be considered discrimin...,A26010,"[affirmative, action, can, be, considered, dis...",0,1,0,3,"[0.0, 0.25, 0.0, 0.75]"
2,A26016,We should ban naturopathy,in favor of,naturopathy is very dangerous for the most vul...,A26016,"[naturopathy, is, very, dangerous, for, the, m...",0,1,1,2,"[0.0, 0.25, 0.25, 0.5]"
3,A26024,We should prohibit women in combat,in favor of,women shouldn't be in combat because they aren...,A26024,"[women, should, n't, be, in, combat, because, ...",0,1,0,0,"[0.0, 1.0, 0.0, 0.0]"
4,A26026,We should ban naturopathy,in favor of,once eradicated illnesses are returning due to...,A26026,"[once, eradicated, illnesses, are, returning, ...",0,1,2,2,"[0.0, 0.2, 0.4, 0.4]"
...,...,...,...,...,...,...,...,...,...,...,...
1571,E07272,We should end mass migration together.,in favor of,Mass migration is a phenomenon that causes dam...,E07272,"[mass, migration, is, a, phenomenon, that, cau...",0,0,1,0,"[0.0, 0.0, 1.0, 0.0]"
1572,E07273,We should end mass migration together.,in favor of,Mass migration hurts the migrants themselves i...,E07273,"[mass, migration, hurts, the, migrants, themse...",0,0,1,2,"[0.0, 0.0, 0.3333333333333333, 0.6666666666666..."
1573,E07275,We should consider Russian interests in the EU...,in favor of,It is neither in the interests of the EU nor R...,E07275,"[it, is, neither, in, the, interests, of, the,...",1,1,1,2,"[0.2, 0.2, 0.2, 0.4]"
1574,E07280,We should adopt an extension of the applicatio...,in favor of,Foreign Policy at the EU level should be based...,E07280,"[foreign, policy, at, the, eu, level, should, ...",1,1,0,2,"[0.25, 0.25, 0.0, 0.5]"


Data cleaning: stopwords removal

In [2]:
import nltk

In [49]:
import numpy as np

In [19]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
from nltk.corpus import stopwords
nltk.download('stopwords')
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
stop_words = set(stopwords.words('english'))

In [13]:
import string

def filter_tokens(tokens):
    filtered_tokens = [token for token in tokens if token.strip() not in stop_words and token.strip() not in string.punctuation and token.strip()]
    return filtered_tokens


In [14]:
test_set["no_stopwords_premise"]=test_set["premise_tokenized"].apply(filter_tokens)

Topic modelling with GENSIM

In [11]:
def get_freq_dist(text):
    freqs = nltk.FreqDist(text)
    return freqs

In [15]:
# we first see the distribution of frequencies of words in the premises to see if we can improve the stop_words list
premises = []
for doc in test_set["no_stopwords_premise"]:
    for token in doc:
        premises.append(token)

premises_freq = get_freq_dist(premises)
premises_freq.most_common(20)

[('people', 350),
 ('women', 160),
 ('naturopathy', 138),
 ('marriage', 133),
 ('flag', 124),
 ('action', 123),
 ('affirmative', 119),
 ("n't", 116),
 ('many', 112),
 ('polygamy', 112),
 ("'s", 109),
 ('home', 108),
 ('burning', 107),
 ('would', 102),
 ('urbanization', 100),
 ('stay', 96),
 ('racial', 94),
 ('profiling', 90),
 ('guantanamo', 90),
 ('men', 88)]

In [16]:
# select from the most frequent words those that seemed too frequent (people) or less informative (would, n't, many and 's)
most_freq_words = ["people", "would", "'n't", "many", "'s"]
custom_stop_words = list(stop_words)
custom_stop_words.append("people")
custom_stop_words = set(custom_stop_words)

In [20]:
# creating the documents from the dataset's premises with no stop_words
docs = [[token for token in doc if token not in custom_stop_words] for doc in test_set["no_stopwords_premise"]]
# lemmatizing the docs
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
# remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]
# remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [22]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token
            )

In [23]:
from gensim.corpora import Dictionary

In [24]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

print("Dictionary lenght 1):", len(dictionary))

# Filter out words that occur less than 5 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=5, no_above=0.5)


print("Dictionary lenght 2):",len(dictionary))

Dictionary lenght 1): 3608
Dictionary lenght 2): 776


In [27]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [28]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 776
Number of documents: 1576


In [29]:
from gensim.models import HdpModel

In [30]:
# We first try with Hierarchical Dirichlet Process to see what would be the optimal number of topics
temp = dictionary[0] 
id2word = dictionary.id2token

hdp = HdpModel(
    corpus=corpus,
    id2word=id2word
)

In [31]:
hdp.print_topics(num_words=10)

[(0,
  '0.008*become + 0.008*much + 0.007*loss + 0.007*ineffective + 0.006*strength + 0.006*expensive + 0.006*gender + 0.006*abandon + 0.006*find + 0.006*right'),
 (1,
  '0.008*usually + 0.007*father + 0.006*nothing + 0.006*current + 0.006*area + 0.006*industry + 0.006*modern + 0.006*look + 0.005*girl + 0.005*place'),
 (2,
  '0.014*cultural + 0.012*overcrowding + 0.010*created + 0.008*activity + 0.008*close + 0.007*market + 0.007*individual + 0.007*american + 0.006*woman + 0.006*allows'),
 (3,
  '0.007*away + 0.006*run + 0.006*often + 0.006*simply + 0.006*racist + 0.006*difference + 0.005*willing + 0.005*natural + 0.005*minor + 0.005*keep'),
 (4,
  '0.007*rural + 0.007*help + 0.007*practice + 0.006*available + 0.006*urban + 0.006*information + 0.006*side + 0.006*consumer + 0.005*risk + 0.005*ability'),
 (5,
  '0.007*technology + 0.007*space + 0.007*position + 0.006*medicine + 0.006*look + 0.006*immoral + 0.006*mental + 0.006*role + 0.006*speech + 0.006*giving'),
 (6,
  '0.012*quality +

Topic modelling with GENSIM in the test-set confirmed the same number of topics previously found in the train-set.

TOPIC MODELLING WITH SKLEARN

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
texts = [text for text in test_set["Premise"]]

In [35]:
# vectorizing with sklearn
# we turn the stop_words set into a list because of parameter restrictions of CountVectorizer
custom_stop_words = list(custom_stop_words)
vectorizer = CountVectorizer(stop_words=custom_stop_words, max_df=0.5, min_df=5,max_features = 7000, ngram_range=(1,2))
tf = vectorizer.fit_transform(texts)

In [36]:
from sklearn.decomposition import LatentDirichletAllocation

In [37]:
lda = LatentDirichletAllocation(n_components=19, max_iter=20,
                                learning_method = 'batch',
                                n_jobs=-1,verbose=0, random_state=42)
lda.fit(tf)

LatentDirichletAllocation(max_iter=20, n_components=19, n_jobs=-1,
                          random_state=42)

In [38]:
def print_top_words(lda, feature_names, n_top_words):
    for topic_idx, topic in enumerate(lda.components_):
        print()
        message = f'Topic {topic_idx}: '
        message += ', '.join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [39]:
n_top_words = 20
tf_feature_names = vectorizer.get_feature_names_out() 
print_top_words(lda, tf_feature_names, n_top_words)


Topic 0: camp, bay, guantanamo, guantanamo bay, society, close, would, place, bay detection, detection camp, detention, detection, india, public, detention camp, bay detention, prisoners, could, need, world

Topic 1: children, polygamy, need, media, woman, family, man, choice, better, society, print, one, support, print media, provide, cannot, help, marriage, much, social

Topic 2: flag, burning, flag burning, country, speech, free, free speech, protected, right, burning flag, banned, every, prohibited, act, compared, deserve, men women, first, weaker, expression

Topic 3: necessary, many, keep, country, naturopathic, want, committed, minority, person, protest, terrorists, medical, example, fact, safe, right, like, problems, medical treatment, burn

Topic 4: rights, human, human rights, right, prohibit, prohibit flag, everyone, eu, everyone right, members, camps, would, countries, tax, migration, universal, declaration human, universal declaration, declaration, disrespectful

Topic 5:

In [40]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [41]:
import pyLDAvis
import pyLDAvis.lda_model

In [42]:
pyLDAvis.enable_notebook()

In [43]:
pyLDAvis.lda_model.prepare(lda,tf,vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16    -0.097721 -0.013168       1        1  8.017294
11    -0.120415  0.007731       2        1  6.791000
7      0.031173 -0.002323       3        1  6.636596
5     -0.193973  0.036312       4        1  6.560764
0      0.092779  0.164950       5        1  6.559431
2      0.002223  0.156303       6        1  6.133496
14    -0.286488 -0.101846       7        1  6.043073
18     0.126567 -0.153992       8        1  5.296482
1     -0.027345 -0.169196       9        1  5.201324
12     0.084600 -0.100328      10        1  5.068215
4      0.012339  0.036947      11        1  4.831351
9      0.131192  0.144436      12        1  4.717831
17    -0.092465  0.173592      13        1  4.713327
6      0.055258 -0.085765      14        1  4.576413
15     0.101894 -0.070611      15        1  4.159253
13     0.022271 -0.054955      16        1  3.751783
3      0.003361  0.081962      17        1  3.742319
10     0.084088 -0.123045      18        1  3.640840
8      0.070662  0.072995      19        1  3.559207, topic_info=            Term        Freq       Total Category  logprob  loglift
874        women  149.000000  149.000000  Default  30.0000  30.0000
520  naturopathy  127.000000  127.000000  Default  29.0000  29.0000
293         flag  116.000000  116.000000  Default  28.0000  28.0000
11        action  115.000000  115.000000  Default  27.0000  27.0000
82       burning  100.000000  100.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
692       rights    5.591642   59.047959  Topic19  -4.4734   0.9786
261        every    4.465888   19.805529  Topic19  -4.6982   1.8461
712     security    4.235756   17.093279  Topic19  -4.7511   1.9405
539          one    4.417113   47.821452  Topic19  -4.7092   0.9536
526        needs    3.949282   25.448471  Topic19  -4.8212   1.4725

[878 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
0         3  0.125037           abandon
0        10  0.187556           abandon
0        12  0.437631           abandon
0        18  0.187556           abandon
1        10  0.228152  abandon marriage
...     ...       ...               ...
892       1  0.717052               yes
892       4  0.179263               yes
893       3  0.706125             young
893       5  0.117687             young
893       8  0.117687             young

[2178 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 12, 8, 6, 1, 3, 15, 19, 2, 13, 5, 10, 18, 7, 16, 14, 4, 11, 9])

In [44]:
# extracting the document topic vectors to use as classification features
doc_topics = lda.transform(tf)
doc_topics

array([[0.0075188 , 0.0075188 , 0.0075188 , ..., 0.86466165, 0.0075188 ,
        0.0075188 ],
       [0.0075188 , 0.0075188 , 0.0075188 , ..., 0.72180451, 0.0075188 ,
        0.0075188 ],
       [0.0037594 , 0.15912601, 0.0037594 , ..., 0.0037594 , 0.0037594 ,
        0.0037594 ],
       ...,
       [0.00219298, 0.00219298, 0.00219298, ..., 0.00219298, 0.00219298,
        0.14093461],
       [0.0075188 , 0.0075188 , 0.0075188 , ..., 0.0075188 , 0.0075188 ,
        0.0075188 ],
       [0.21198654, 0.00877193, 0.00877193, ..., 0.00877193, 0.00877193,
        0.00877193]])

In [45]:
%store -r doc2vec_final

In [46]:
%store -r tagged_docs

In [47]:
# we transform the doc2vec object in a numpy array of vectors
doc2vec_vec = []
for i in range(len(tagged_docs)):
    doc2vec_vec.append(doc2vec_final[i])

In [50]:
# we can now concatenate the topic probability vectors and the doc2vec vectors for each document

doc_topic_vec = np.array([np.concatenate([vec1, vec2]) for vec1, vec2 in zip(doc_topics, doc2vec_vec)])
len(doc_topic_vec)

1576

In [52]:
print(doc_topic_vec)

[[ 0.0075188   0.0075188   0.0075188  ... -0.00119576 -0.00475505
   0.00225294]
 [ 0.0075188   0.0075188   0.0075188  ... -0.00372379 -0.00125304
  -0.00277493]
 [ 0.0037594   0.15912601  0.0037594  ... -0.00434601  0.00148076
  -0.00333795]
 ...
 [ 0.00219298  0.00219298  0.00219298 ... -0.02795893 -0.08207933
   0.03142953]
 [ 0.0075188   0.0075188   0.0075188  ... -0.02316896 -0.08807217
   0.03245522]
 [ 0.21198654  0.00877193  0.00877193 ... -0.02547694 -0.08931737
   0.03672359]]


In [56]:
X_test=doc_topic_vec

In [57]:
y_test=test_set.general_label